# Solar Panel Dataset
## Company: EasyGreen
### Explanation of Variables in Dataset

totalUsePower: Total kWh (amount of electricity) used today.


totalProductPower: Total kWh (amount of electricity) produced by the solar panels today.


totalSelfUsePower: Total kWh (amount of electricity) of today's production used for home electricity consumption.


totalBuyPower: Total kWh (amount of electricity) bought from the grid today.

In [12]:
from sqlalchemy import create_engine
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from pyairtable import Api
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Connection details
username = 'introToML'
password = 'pddbb1c530308955bbd2c5aa59a647e30074fc7b7ebde93f626c481f4de93f7de'
host = 'ec2-63-32-137-56.eu-west-1.compute.amazonaws.com'
port = '5432'
dbname = 'dcsdgo449thll5'

# Connection string
connection_string = f'postgresql+psycopg2://{username}:{password}@{host}:{port}/{dbname}'

# Create the engine
engine = create_engine(connection_string)

# Specify your table name
table_name = 'power_usage'

# Read the table into a DataFrame
df = pd.read_sql_table(table_name, engine, schema='public')

In [6]:
addressData = pd.read_csv('addressData.csv')
# rename Kunde ID to user_id, Kundens navn to name, CPR adresse to address
addressData.rename(columns={'Kunde ID':'user_id', 'Kundens navn':'name', 'CPR adresse':'address'}, inplace=True)
addressData['user_id'] = addressData['user_id'].fillna(0).astype(int)

In [11]:
ageData = pd.read_csv('../data/ageData.csv')
# create a new column with the age of the user calculated by Fødseldato (date of birth)
ageData['Fødselsdato'] = ageData['Fødselsdato'].fillna(0)
ageData['Fødselsdato'] = pd.to_datetime(ageData['Fødselsdato'], errors='coerce')
ageData['Alder'] = 2024 - ageData['Fødselsdato'].dt.year

# convert age to int
ageData['Alder'] = ageData['Alder'].fillna(0).astype(int)
ageData['Kunde ID'] = ageData['Kunde ID'].fillna(0).astype(int)

# rename Kunde ID to user_id, Alder to age
ageData.rename(columns={'Kunde ID':'user_id', 'Alder':'age'}, inplace=True)

ageData.tail(20)

,Kundens navn,user_id,Fødselsdato,age
1032,Kim Grevelund,1913,1972-03-24,52
1033,Hildur Annika Jacobsen,1914,1970-02-09,54
1034,Kim Pedersen,1915,2024-04-14,0
1035,Morten Vittrup,1916,1965-08-13,59
1036,Per Bay Larsen,1917,1967-03-05,57
1037,Leif Andreas,1918,2024-07-04,0
1038,Thomas Blem,1919,1969-07-02,55
1039,Marianne Helt,1920,1961-09-19,63
1040,Sebbe Bull,1921,1987-03-06,37
1041,Henrik Larsen,1923,1953-05-28,71


In [7]:
def clean_addresses(address):
    # Replace unwanted characters with a space or remove them
    address = address.str.replace(r'[@#*]', '', regex=True)  # Remove @, #, *
    address = address.str.replace(r'[\n\r]', ' ', regex=True)  # Replace newlines with space
    address = address.str.strip()  # Strip leading/trailing whitespace
    return address

# Apply cleaning function
addressData['address'] = clean_addresses(addressData['address'])

In [19]:
import requests
import time 

def geocode_address(api_key, address):
    base_url = "https://us1.locationiq.com/v1/search.php"
    params = {
        "key": api_key,
        "q": address,
        "format": "json"
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        if len(data) > 0:
            lat = data[0]['lat']
            lon = data[0]['lon']
            print(f"Address: {address} -> Latitude: {lat}, Longitude: {lon}")
            time.sleep(1)
            return lat, lon
        else:
            print(f"Address: {address} -> No results found.")
            return None, None
        
    else:
        print(f"Address: {address} -> Failed to geocode with status code {response.status_code}.")
        time.sleep(1)
        return None, None    
        
def apply_geocoding(row):
    lat, lon = geocode_address("pk.fdf56d0c2c0bbfc3b4054a3a45c0bd72", row['address'])
    return pd.Series([lat, lon])

In [ ]:
# Applying geocoding
addressData[['latitude', 'longitude']] = addressData.apply(apply_geocoding, axis=1)

addressData.head()

In [22]:
addressData.to_csv('addressData.csv', index=False)

In [19]:
# merge addressData and df on user_id

#dfMerged = df.merge(addressData, on='user_id', how='left')
df = pd.read_csv('../data/dfMerged.csv')
dfMerged = df.merge(ageData, on='user_id', how='left')

dfMerged.drop(columns=['dailychargecapacity', 'dailydischargecapacity', 'name', 'address', 'Kundens navn', 'Fødselsdato', 'plantId'], inplace=True)

dfMerged.to_csv('../data/dfMerged.csv', index=False)

In [26]:
dfMerged.to_csv('dfMerged.csv', index=False)

In [2]:
mapData = pd.read_csv('../data/dfMerged.csv')

In [6]:
import matplotlib
import matplotlib.pyplot as plt
cmap = plt.cm.get_cmap('viridis')  # You can choose any available colormap
norm = plt.Normalize(mapData['totalProductPower'].min(), mapData['totalProductPower'].max())
mapData['colorProductPower'] = mapData['totalProductPower'].apply(lambda x: matplotlib.colors.rgb2hex(cmap(norm(x))))


/var/folders/r7/5qjvym7502z0hk7l29b80j640000gn/T/ipykernel_67321/2371745365.py:3: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = plt.cm.get_cmap('viridis')  # You can choose any available colormap


,user_id,usage_date,totalBuyPower,totalProductPower,totalUsePower,totalOnGridPower,night_usage,totalSelfUsePower,latitude,longitude,age,colorProductPower
0,909,2022-11-02,9.11,0.00,9.11,0.00,6.663083,0.00,56.084327,12.456106,52.0,#440154
1,909,2022-11-03,51.74,0.00,52.93,0.00,41.646500,0.00,56.084327,12.456106,52.0,#440154
2,909,2022-11-04,42.67,3.08,45.96,0.00,40.036917,3.08,56.084327,12.456106,52.0,#440154
3,909,2022-11-05,15.04,29.07,34.60,9.36,13.077167,19.71,56.084327,12.456106,52.0,#440256
4,909,2022-11-06,21.33,3.03,24.52,0.10,10.748500,2.93,56.084327,12.456106,52.0,#440154
...,...,...,...,...,...,...,...,...,...,...,...,...
394739,1807,2024-04-11,0.10,8.39,8.08,0.05,4.007750,8.34,55.728342,9.107490,47.0,#440154
394740,1811,2024-04-11,22.32,3.43,29.49,0.01,6.495000,3.42,55.635639,12.637176,82.0,#440154
394741,1816,2024-04-11,30.29,13.02,37.70,5.43,33.714333,7.59,55.719370,11.379678,53.0,#440154
394742,1818,2024-04-11,11.28,9.42,24.36,0.00,17.644833,9.42,55.479497,9.005815,63.0,#440154


In [8]:
mapData.to_csv('../data/mapData.csv', index=False)